In [ ]:
import random
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import gymnasium as gym
from tqdm import tqdm # For progress bars during evolution

### Chromosome
This section defines the Chromosome class, which represents a neural network's structure and parameters. It handles node and edge management, network operations (like forward pass), structural mutations such as node division, and visualization. It also contains an inner class for running the neural network.

In [ ]:
"""
This module defines the Chromosome class, which represents a neural network's
structure and parameters for neuroevolutionary algorithms. It handles node and
edge management, network operations, structural mutations like node division,
and visualization. It also contains an inner class for running the neural network.
"""

class Chromosome:
    """
    Represents a neural network chromosome, encapsulating its nodes, edges,
    and methods for genetic operations and network execution.

    A chromosome's structure is defined by two Pandas DataFrames:
    - `nodes`: Stores information about each neuron (ID, type, bias, birth generation, division count).
    - `edges`: Stores information about connections between neurons (ID, source, target, weight, enabled status).
    """
    def __init__(
        self,
        id: int,
        nodes_df: pd.DataFrame = None,
        edges_df: pd.DataFrame = None,
        inputs: int = None,
        outputs: int = None,
        hidden_nodes: int = None,  # Number of hidden nodes to create if starting from scratch
        connectivity_ratio: float = 0.5,  # Ratio of possible connections to create for initial network
        generation: int = 0,       # Track the generation of this chromosome
    ):
        """
        Initializes a Chromosome instance.

        Can initialize with pre-existing node and edge DataFrames, or create a
        new, randomly structured network based on specified input/output/hidden
        node counts and connectivity ratio.

        Args:
            id (int): Unique identifier for this chromosome.
            nodes_df (pd.DataFrame, optional): DataFrame containing node data.
                If provided, `edges_df` must also be provided. Defaults to None.
            edges_df (pd.DataFrame, optional): DataFrame containing edge data.
                If provided, `nodes_df` must also be provided. Defaults to None.
            inputs (int, optional): Number of input nodes to create. Required if
                `nodes_df` and `edges_df` are None.
            outputs (int, optional): Number of output nodes to create. Required if
                `nodes_df` and `edges_df` are None.
            hidden_nodes (int, optional): Initial number of hidden nodes to create.
                Defaults to 10 if not specified and creating a new network.
            connectivity_ratio (float, optional): Density of initial connections.
                A ratio of 0.5 means roughly half of possible connections between
                layers (input-hidden, hidden-output, hidden-hidden) will be made.
                Defaults to 0.5.
            generation (int, optional): The generation this chromosome was created in.
                Defaults to 0.

        Raises:
            ValueError: If `nodes_df` is provided but `edges_df` is not, or vice-versa.
            ValueError: If creating a new network and `inputs` or `outputs` are not provided.
        """
        self.id = id
        self.generation = generation

        if nodes_df is None and edges_df is None:
            if inputs is None or outputs is None:
                raise ValueError("Inputs and outputs must be specified when creating a new chromosome.")

            self.nodes = pd.DataFrame(columns=['id', 'type', 'bias', 'birth_generation', 'division_count'])
            self.edges = pd.DataFrame(columns=['id', 'source', 'target', 'weight', 'enabled'])

            if connectivity_ratio is None:
                connectivity_ratio = 0.5
            
            # Default hidden_nodes if not specified
            if hidden_nodes is None:
                hidden_nodes = 10

            node_count = 0
            edge_count = 0

            # Create input nodes
            input_start_id = node_count
            for i in range(inputs):
                self.nodes.loc[node_count] = {
                    'id': node_count,
                    'type': 'input',
                    'bias': 0.0,  # Input nodes typically don't have bias
                    'birth_generation': generation,
                    'division_count': 0
                }
                node_count += 1

            # Create hidden nodes
            hidden_start_id = node_count
            for h in range(hidden_nodes):
                self.nodes.loc[node_count] = {
                    'id': node_count,
                    'type': 'hidden',
                    'bias': np.random.uniform(-1, 1),
                    'birth_generation': generation,
                    'division_count': 0
                }
                node_count += 1

            # Create output nodes
            output_start_id = node_count
            for o in range(outputs):
                self.nodes.loc[node_count] = {
                    'id': node_count,
                    'type': 'output',
                    'bias': np.random.uniform(-1, 1),
                    'birth_generation': generation,
                    'division_count': 0
                }
                node_count += 1

            input_ids = list(range(input_start_id, hidden_start_id))
            hidden_ids = list(range(hidden_start_id, output_start_id))
            output_ids = list(range(output_start_id, node_count))
            
            # Helper for initial network creation (local scope)
            def _would_create_cycle_initial(source, target, existing_edges_df):
                """Check if adding edge would create a cycle in a temporary graph."""
                G_temp = nx.DiGraph()
                for _, edge_row in existing_edges_df.iterrows():
                    if edge_row['enabled']:
                        G_temp.add_edge(edge_row['source'], edge_row['target'])
                G_temp.add_edge(source, target)
                try:
                    nx.find_cycle(G_temp)
                    return True
                except nx.NetworkXNoCycle:
                    return False

            # Connect inputs to some hidden nodes
            if hidden_ids: # Ensure there are hidden nodes to connect to
                for i in input_ids:
                    num_connections = max(1, int(connectivity_ratio * len(hidden_ids)))
                    targets = random.sample(hidden_ids, min(num_connections, len(hidden_ids))) # Ensure not to sample more than available
                    for t in targets:
                        self.edges.loc[edge_count] = {
                            'id': edge_count,
                            'source': i,
                            'target': t,
                            'weight': np.random.uniform(-1, 1),
                            'enabled': True
                        }
                        edge_count += 1

            # Connect hidden nodes to output nodes
            if hidden_ids and output_ids: # Ensure both hidden and output nodes exist
                for h in hidden_ids:
                    num_connections = max(1, int(connectivity_ratio * len(output_ids)))
                    targets = random.sample(output_ids, min(num_connections, len(output_ids)))
                    for t in targets:
                        self.edges.loc[edge_count] = {
                            'id': edge_count,
                            'source': h,
                            'target': t,
                            'weight': np.random.uniform(-1, 1),
                            'enabled': True
                        }
                        edge_count += 1
            elif not hidden_ids and input_ids and output_ids: # Direct input to output if no hidden nodes
                for i in input_ids:
                    num_connections = max(1, int(connectivity_ratio * len(output_ids)))
                    targets = random.sample(output_ids, min(num_connections, len(output_ids)))
                    for t in targets:
                         self.edges.loc[edge_count] = {
                            'id': edge_count,
                            'source': i,
                            'target': t,
                            'weight': np.random.uniform(-1, 1),
                            'enabled': True
                        }
                         edge_count += 1

            # Add some additional hidden-to-hidden connections
            if len(hidden_ids) > 1:
                possible_h2h = [(src, tgt) for src in hidden_ids for tgt in hidden_ids if src != tgt]
                random.shuffle(possible_h2h)
                
                # Limit connections to avoid excessively dense networks
                num_h2h_to_try = int(connectivity_ratio * len(possible_h2h))
                
                for src, tgt in possible_h2h[:num_h2h_to_try]:
                    # Create a temporary DataFrame to test cycle prevention with current edges
                    temp_edges_for_cycle_check = self.edges.copy()
                    if not _would_create_cycle_initial(src, tgt, temp_edges_for_cycle_check):
                        self.edges.loc[edge_count] = {
                            'id': edge_count,
                            'source': src,
                            'target': tgt,
                            'weight': np.random.uniform(-1, 1),
                            'enabled': True
                        }
                        edge_count += 1
        elif nodes_df is not None and edges_df is not None:
            self.nodes = nodes_df.copy()
            self.edges = edges_df.copy()
        else:
            raise ValueError("Both nodes_df and edges_df must be provided, or neither.")
        
        # Initialize the NN instance for running the network
        self.NN = self.NeuralNetwork(self)
    
    # Node-related methods
    def get_node_by_id(self, node_id: int) -> pd.Series | None:
        """
        Retrieves a node by its unique identifier.

        Args:
            node_id (int): The ID of the node to retrieve.

        Returns:
            pd.Series or None: A Pandas Series representing the node if found, otherwise None.
        """
        result = self.nodes[self.nodes['id'] == node_id]
        return result.iloc[0] if not result.empty else None
    
    def get_nodes_by_type(self, node_type: str) -> pd.DataFrame:
        """
        Retrieves all nodes of a specific type.

        Args:
            node_type (str): The type of node to filter by (e.g., 'input', 'hidden', 'output').

        Returns:
            pd.DataFrame: A DataFrame containing all nodes of the specified type.
        """
        return self.nodes[self.nodes['type'] == node_type]
    
    def get_input_nodes(self) -> pd.DataFrame:
        """
        Retrieves all input nodes.

        Returns:
            pd.DataFrame: A DataFrame containing all input nodes.
        """
        return self.get_nodes_by_type('input')
    
    def get_output_nodes(self) -> pd.DataFrame:
        """
        Retrieves all output nodes.

        Returns:
            pd.DataFrame: A DataFrame containing all output nodes.
        """
        return self.get_nodes_by_type('output')
    
    def get_hidden_nodes(self) -> pd.DataFrame:
        """
        Retrieves all hidden nodes.

        Returns:
            pd.DataFrame: A DataFrame containing all hidden nodes.
        """
        return self.get_nodes_by_type('hidden')
    
    # Edge-related methods
    def get_edge_by_id(self, edge_id: int) -> pd.Series | None:
        """
        Retrieves an edge by its unique identifier.

        Args:
            edge_id (int): The ID of the edge to retrieve.

        Returns:
            pd.Series or None: A Pandas Series representing the edge if found, otherwise None.
        """
        result = self.edges[self.edges['id'] == edge_id]
        return result.iloc[0] if not result.empty else None
    
    def get_edges_by_source(self, source_id: int) -> pd.DataFrame:
        """
        Retrieves all edges originating from a specific source node.

        Args:
            source_id (int): The ID of the source node.

        Returns:
            pd.DataFrame: A DataFrame containing all outgoing edges from the source node.
        """
        return self.edges[self.edges['source'] == source_id]
    
    def get_edges_by_target(self, target_id: int) -> pd.DataFrame:
        """
        Retrieves all edges terminating at a specific target node.

        Args:
            target_id (int): The ID of the target node.

        Returns:
            pd.DataFrame: A DataFrame containing all incoming edges to the target node.
        """
        return self.edges[self.edges['target'] == target_id]
    
    def get_edges_between(self, source_id: int, target_id: int) -> pd.DataFrame:
        """
        Retrieves all edges connecting a specific source node to a specific target node.

        Args:
            source_id (int): The ID of the source node.
            target_id (int): The ID of the target node.

        Returns:
            pd.DataFrame: A DataFrame containing edges between the specified source and target.
        """
        mask = (self.edges['source'] == source_id) & (self.edges['target'] == target_id)
        return self.edges[mask]
    
    def get_enabled_edges(self) -> pd.DataFrame:
        """
        Retrieves all enabled edges in the network.

        Returns:
            pd.DataFrame: A DataFrame containing all enabled edges.
        """
        return self.edges[self.edges['enabled'] == True]
    
    def would_create_cycle(self, source: int, target: int) -> bool:
        """
        Checks if adding an edge from a source node to a target node would create a cycle
        in the current network's enabled connections.

        Args:
            source (int): The ID of the source node for the proposed edge.
            target (int): The ID of the target node for the proposed edge.

        Returns:
            bool: True if adding the edge would create a cycle, False otherwise.
        """
        G = nx.DiGraph()
        for _, edge in self.edges.iterrows():
            if edge['enabled']:
                G.add_edge(edge['source'], edge['target'])
        
        G.add_edge(source, target) # Temporarily add the proposed edge for cycle check
        
        try:
            nx.find_cycle(G)
            return True  # Found a cycle
        except nx.NetworkXNoCycle:
            return False  # No cycle found
            
    # Network modification operations
    def add_node(self, node_type: str, bias: float = None, birth_generation: int = None) -> int:
        """
        Adds a new node to the network.

        The new node ID is automatically determined as the next available integer.
        Bias is initialized randomly for hidden/output nodes if not provided, and 0.0 for input nodes.

        Args:
            node_type (str): The type of the node ('input', 'hidden', 'output').
            bias (float, optional): The bias value for the new node. If None, it's
                                    randomly generated (or 0 for input nodes). Defaults to None.
            birth_generation (int, optional): The generation in which this node was created.
                                            If None, uses the chromosome's current generation. Defaults to None.

        Returns:
            int: The ID of the newly created node.
        """
        # Get new node_id
        new_node_id = int(self.nodes['id'].max() + 1) if not self.nodes.empty else 0
        
        # Set bias based on node type
        if bias is None:
            if node_type == 'input':
                bias = 0.0
            else:
                bias = np.random.uniform(-1, 1)
        
        # If birth_generation not provided, use current generation
        if birth_generation is None:
            birth_generation = self.generation
        
        new_node_data = {
            'id': new_node_id,
            'type': node_type,
            'bias': bias,
            'birth_generation': birth_generation,
            'division_count': 0
        }
        
        # Append to nodes DataFrame
        # Using pd.concat for immutability and better practices in Pandas
        self.nodes = pd.concat([self.nodes, pd.DataFrame([new_node_data])], ignore_index=True)
        
        return new_node_id
    
    def add_edge(self, source: int, target: int, weight: float = None, enabled: bool = True) -> int | None:
        """
        Adds a new edge to the network, preventing cycles.

        If the edge would create a cycle with existing enabled edges, it is not added.
        Weight is generated randomly if not provided.

        Args:
            source (int): The ID of the source node.
            target (int): The ID of the target node.
            weight (float, optional): The weight of the new edge. If None, it's
                                    randomly generated. Defaults to None.
            enabled (bool, optional): Whether the edge is initially enabled. Defaults to True.

        Returns:
            int or None: The ID of the newly created edge if added, None if it would create a cycle.
        """
        # Check if adding this edge would create a cycle
        if self.would_create_cycle(source, target):
            return None
            
        # Generate weight if not provided
        if weight is None:
            weight = np.random.uniform(-1, 1)
        
        # Get new edge_id
        new_edge_id = int(self.edges['id'].max() + 1) if not self.edges.empty else 0
        
        new_edge_data = {
            'id': new_edge_id,
            'source': source,
            'target': target,
            'weight': weight,
            'enabled': enabled
        }
        
        # Append to edges DataFrame
        self.edges = pd.concat([self.edges, pd.DataFrame([new_edge_data])], ignore_index=True)
        
        return new_edge_id
    
    def disable_edge(self, edge_id: int) -> bool:
        """
        Disables an existing edge by setting its 'enabled' status to False.

        Args:
            edge_id (int): The ID of the edge to disable.

        Returns:
            bool: True if the edge was found and disabled, False otherwise.
        """
        idx = self.edges.index[self.edges['id'] == edge_id].tolist()
        if idx:
            self.edges.loc[idx[0], 'enabled'] = False
            return True
        return False
    
    def update_node_bias(self, node_id: int, new_bias: float) -> bool:
        """
        Updates the bias of a specific node.

        Args:
            node_id (int): The ID of the node whose bias is to be updated.
            new_bias (float): The new bias value.

        Returns:
            bool: True if the node was found and updated, False otherwise.
        """
        idx = self.nodes.index[self.nodes['id'] == node_id].tolist()
        if idx:
            self.nodes.loc[idx[0], 'bias'] = new_bias
            return True
        return False
    
    def update_edge_weight(self, edge_id: int, new_weight: float) -> bool:
        """
        Updates the weight of a specific edge.

        Args:
            edge_id (int): The ID of the edge whose weight is to be updated.
            new_weight (float): The new weight value.

        Returns:
            bool: True if the edge was found and updated, False otherwise.
        """
        idx = self.edges.index[self.edges['id'] == edge_id].tolist()
        if idx:
            self.edges.loc[idx[0], 'weight'] = new_weight
            return True
        return False
    
    def get_connection_count(self, node_id: int) -> int:
        """
        Calculates the total number of incoming and outgoing connections for a given node.

        Args:
            node_id (int): The ID of the node.

        Returns:
            int: The sum of incoming and outgoing connections for the node.
        """
        incoming = len(self.get_edges_by_target(node_id))
        outgoing = len(self.get_edges_by_source(node_id))
        return incoming + outgoing
        
    # Node Division (Structural Mutation) methods
    def evaluate_division(self, node_id: int, division_model: 'Chromosome') -> bool:
        """
        Evaluates whether a hidden node should divide based on inputs to a separate
        neural network (the `division_model`). Adds randomness to introduce variability.

        This method is designed to be used with a `division_model` that has 4 inputs:
        [normalized_age, normalized_division_count, normalized_connection_count, normalized_node_id].
        Its single output determines the division probability.

        Args:
            node_id (int): The ID of the node to evaluate for division.
            division_model (Chromosome): A separate Chromosome instance acting as a
                                         neural network to make the division decision.

        Returns:
            bool: True if the node should divide, False otherwise.
        """
        node = self.get_node_by_id(node_id)
        
        if node is None or node['type'] != 'hidden':
            return False # Only hidden nodes can divide
        
        node_age = self.generation - node['birth_generation']
        division_count = node['division_count']
        connection_count = self.get_connection_count(node_id)
        
        # Arbitrary maximum values for normalization
        max_age = 20
        max_division = 5
        max_connections = 20
        # Determine max_node_id dynamically to scale better with network growth
        max_node_id = self.nodes['id'].max() + 1 if not self.nodes.empty else 100 
        
        normalized_age = node_age / max_age
        normalized_division = division_count / max_division
        normalized_connections = connection_count / max_connections
        normalized_node_id = node_id / max_node_id
        
        # Add random noise to inputs to create variability
        noise_range = 0.3
        noisy_age = normalized_age + random.uniform(-noise_range, noise_range)
        noisy_division = normalized_division + random.uniform(-noise_range, noise_range)
        noisy_connections = normalized_connections + random.uniform(-noise_range, noise_range)
        noisy_node_id = normalized_node_id + random.uniform(-noise_range, noise_range)
        
        inputs_for_division_model = [noisy_age, noisy_division, noisy_connections, noisy_node_id]
        output_from_model = division_model.NN.run(inputs_for_division_model)
        
        # Add node-specific randomness to the output
        node_specific_factor = random.uniform(0.7, 1.3)
        modified_output = output_from_model[0] * node_specific_factor
        
        # Use a random threshold for the decision
        threshold = random.uniform(0.2, 0.5)
        
        return modified_output > threshold
    
    def perform_division(self, node_id: int, bias_func=None) -> list[int]:
        """
        Performs the division of a single hidden node.
        A new "daughter" node is created, and connections are established.
        The parent node's division count is incremented.

        Args:
            node_id (int): The ID of the parent node to divide.
            bias_func (callable, optional): A function that takes the parent's bias
                                           and returns the daughter's bias. If None,
                                           a default random variation is used. Defaults to None.

        Returns:
            list[int]: A list containing the ID of the newly created daughter node.
                       Returns an empty list if the parent node is not hidden.
        """
        parent_node = self.get_node_by_id(node_id)
        
        if parent_node is None or parent_node['type'] != 'hidden':
            return []
        
        # Increment division count of parent node
        parent_idx = self.nodes.index[self.nodes['id'] == node_id].tolist()[0]
        self.nodes.at[parent_idx, 'division_count'] += 1
        
        # Default bias function: random variation of parent bias
        if bias_func is None:
            bias_func = lambda p_bias: p_bias + np.random.uniform(-0.2, 0.2)
        
        # Create a daughter node of the same type ('hidden')
        daughter_id = self.add_node(
            node_type='hidden',
            bias=bias_func(parent_node['bias']),
            birth_generation=self.generation
        )
        
        # Connect parent to daughter if it doesn't create a cycle
        if daughter_id is not None and not self.would_create_cycle(node_id, daughter_id):
            self.add_edge(source=node_id, target=daughter_id)
        
        # Connect incoming edges to daughter (replicate parent's incoming connections)
        for _, edge in self.get_edges_by_target(node_id).iterrows():
            # Skip self-loops and connections from parent (to avoid creating loops)
            if edge['source'] != node_id and edge['source'] != daughter_id:
                if not self.would_create_cycle(edge['source'], daughter_id):
                    self.add_edge(source=edge['source'], target=daughter_id, weight=edge['weight'])
        
        # Connect daughter to outgoing edges (replicate parent's outgoing connections)
        for _, edge in self.get_edges_by_source(node_id).iterrows():
            # Skip self-loops and connections to parent (to avoid creating loops)
            if edge['target'] != node_id and edge['target'] != daughter_id:
                if not self.would_create_cycle(daughter_id, edge['target']):
                    self.add_edge(source=daughter_id, target=edge['target'], weight=edge['weight'])
        
        return [daughter_id] if daughter_id is not None else []
    
    def evaluate_all_nodes_for_division(self, division_model: 'Chromosome', bias_func=None) -> list[int]:
        """
        Evaluates all hidden nodes in the chromosome for division and performs the division
        if the criteria set by the `division_model` are met.

        Args:
            division_model (Chromosome): A Chromosome instance that acts as the decision model
                                         for node division.
            bias_func (callable, optional): Function to calculate the bias of daughter nodes.
                                           If None, a default is used. Defaults to None.

        Returns:
            list[int]: A list of IDs of all newly created nodes during this process.
        """
        hidden_nodes = self.get_hidden_nodes()
        new_nodes = []
        
        # Make a copy of node IDs to avoid iteration issues when adding nodes
        nodes_to_evaluate = hidden_nodes['id'].tolist()
        
        for node_id in nodes_to_evaluate:
            # Check if node_id still exists after previous divisions (important if new nodes are added)
            # This handles cases where a node might be a newly divided node itself and should not divide again immediately
            if node_id in self.nodes['id'].values and self.get_node_by_id(node_id)['type'] == 'hidden':
                if self.evaluate_division(node_id, division_model):
                    daughter_nodes = self.perform_division(node_id, bias_func)
                    new_nodes.extend(daughter_nodes)
        
        return new_nodes
    
    # Visualization methods
    def to_networkx(self) -> nx.DiGraph:
        """
        Converts the chromosome's network structure into a NetworkX directed graph object.
        This is useful for graph analysis and visualization.

        Returns:
            nx.DiGraph: A NetworkX directed graph representing the chromosome.
        """
        G = nx.DiGraph()
        
        # Add nodes with their attributes
        for _, node in self.nodes.iterrows():
            G.add_node(
                node['id'], 
                type=node['type'],
                bias=node['bias'],
                birth_generation=node['birth_generation'],
                division_count=node['division_count']
            )
        
        # Add enabled edges with their attributes
        for _, edge in self.edges.iterrows():
            if edge['enabled']:
                G.add_edge(
                    edge['source'],
                    edge['target'],
                    weight=edge['weight'],
                    id=edge['id']
                )
        
        return G
    
    def visualize(self, ax=None, figsize: tuple[int, int] = (10, 8)):
        """
        Visualizes the neural network structure using NetworkX and Matplotlib.

        Nodes are colored based on their type (input, hidden, output).
        Edges are colored based on weight sign (green for positive, red for negative)
        and width is proportional to absolute weight.

        Args:
            ax (matplotlib.axes.Axes, optional): A Matplotlib axes object to draw on.
                                                If None, a new figure and axes are created.
                                                Defaults to None.
            figsize (tuple[int, int], optional): Dimensions of the figure if `ax` is None.
                                               Defaults to (10, 8).

        Returns:
            matplotlib.axes.Axes: The Matplotlib axes object the graph was drawn on.
        """
        if ax is None:
            fig, ax = plt.subplots(figsize=figsize)
        
        G = self.to_networkx()
        
        # Use a hierarchical layout (e.g., 'dot' from Graphviz) if available, otherwise fall back
        try:
            pos = nx.nx_agraph.graphviz_layout(G, prog='dot')
        except ImportError:
            # Fall back to spring layout if graphviz/pygraphviz is not installed
            print("Graphviz not found. Falling back to spring layout for visualization. For better layouts, install graphviz and pygraphviz.")
            pos = nx.spring_layout(G, k=0.8, iterations=50) # Increased k for more spread, more iterations for stability
        
        # Draw nodes by type with different colors
        node_colors = []
        for node_id in G.nodes():
            node_type = G.nodes[node_id]['type']
            if node_type == 'input':
                node_colors.append('lightblue')
            elif node_type == 'hidden':
                node_colors.append('lightgreen')  
            elif node_type == 'output':
                node_colors.append('lightcoral')
        
        nx.draw_networkx_nodes(G, pos, ax=ax, node_color=node_colors, node_size=700, alpha=0.9) # Slightly larger nodes
        
        edge_colors = []
        edge_widths = []
        
        for u, v, data in G.edges(data=True):
            weight = data['weight']
            edge_colors.append('green' if weight >= 0 else 'red') # Green for positive, red for negative
            edge_widths.append(abs(weight) * 2 + 0.5) # Minimum width for visibility
        
        nx.draw_networkx_edges(G, pos, ax=ax, edge_color=edge_colors, width=edge_widths, 
                               arrowsize=15, connectionstyle='arc3,rad=0.1', alpha=0.7)
        
        # Draw node labels (node IDs)
        node_labels = {node_id: node_id for node_id in G.nodes()}
        nx.draw_networkx_labels(G, pos, ax=ax, labels=node_labels, font_size=9, font_weight='bold')

        # Draw edge labels (weights) for enabled edges
        edge_labels = {
            (u, v): f"{data['weight']:.2f}"
            for u, v, data in G.edges(data=True) if data.get('enabled', True)
        }
        nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, ax=ax, font_size=7, bbox={"alpha": 0.7, "pad": 0.5})
        
        if ax is None: # Only set title/axis if a new figure was created
            plt.title(f"Chromosome {self.id} Network Structure", size=14)
            plt.axis('off')
            plt.tight_layout()
            plt.show()
        
        return ax
    
    class NeuralNetwork:
        """
        An inner class representing the executable neural network defined by the Chromosome's
        nodes and edges. Handles the forward pass computation.
        """
        def __init__(self, chromosome_instance: 'Chromosome'):
            """
            Initializes the NeuralNetwork executor.

            Args:
                chromosome_instance (Chromosome): The parent Chromosome instance whose
                                                  nodes and edges define this network.
            """
            self.edges = chromosome_instance.edges
            self.nodes = chromosome_instance.nodes
            self.results = pd.DataFrame({'node id': self.nodes['id'].values, 'result': None})
        
        def run(self, X: np.ndarray | list) -> list[float]:
            """
            Performs a forward pass through the neural network given an input array.

            This method topologically sorts the graph for efficient execution,
            computes node activations using ReLU, and returns the outputs of
            the output nodes. Handles cycles by falling back to a default order.

            Args:
                X (np.ndarray | list): The input values for the input nodes.
                                      Must match the number of input nodes.

            Returns:
                list[float]: A list of activation values for the output nodes.
                             Any uncomputed output node results will be 0.0.

            Raises:
                ValueError: If the input array `X` does not match the number of input nodes.
            """
            # Reset results for a new run
            self.results['result'] = None

            inp_ids = self.nodes.loc[self.nodes['type'] == 'input', 'id'].values
            
            if len(X) != len(inp_ids):
                raise ValueError(f"Input size {len(X)} doesn't match network input nodes {len(inp_ids)}")
                
            # Set input values in the results DataFrame
            for i, val in enumerate(X):
                self.results.loc[self.results['node id'] == inp_ids[i], 'result'] = val
            
            # Create a directed graph for topological sort
            G_run = nx.DiGraph()
            for _, edge in self.edges.iterrows():
                if edge['enabled']:
                    G_run.add_edge(edge['source'], edge['target'])
            
            execution_order = []
            try:
                # Get execution order through topological sort
                execution_order = list(nx.topological_sort(G_run))
                # Filter out input nodes as their values are already set
                execution_order = [node_id for node_id in execution_order 
                                   if node_id not in inp_ids]
            except nx.NetworkXUnfeasible:
                # If there's a cycle, fall back to default order (e.g., node ID order for non-inputs)
                # This might not process nodes in the 'correct' order for cyclic graphs,
                # but prevents crashes and allows the network to still attempt computation.
                print("Warning: Cycle detected in network. Falling back to default execution order.")
                execution_order = self.nodes.loc[~self.nodes['id'].isin(inp_ids), 'id'].values
            
            def ReLU(x: float) -> float:
                """Rectified Linear Unit activation function."""
                return np.maximum(0, x)
            
            # Execute nodes in order
            for node_id in execution_order:
                # Only process nodes if all their necessary inputs are computed
                incoming_edges = self.edges.loc[(self.edges['target'] == node_id) & (self.edges['enabled'] == True)]
                
                if incoming_edges.empty:
                    # If a node has no incoming edges (and is not an input), its value remains None or 0.0
                    # This handles isolated hidden/output nodes, or nodes that are targets of only disabled edges.
                    continue

                source_node_ids = incoming_edges['source'].values
                # Get results for source nodes that have already been computed
                source_vals_df = self.results.loc[self.results['node id'].isin(source_node_ids)]
                
                # Check if all required source node values are available (not None/NaN)
                if not source_vals_df.empty and not source_vals_df['result'].isnull().any():
                    # Align weights with source values
                    # Create a mapping from source_id to its computed result
                    source_results_map = source_vals_df.set_index('node id')['result'].to_dict()
                    
                    # Ensure weights are aligned with the order of source_values
                    # Filter incoming edges to only those whose source results are available
                    valid_incoming_edges = incoming_edges[incoming_edges['source'].isin(source_results_map.keys())]

                    if not valid_incoming_edges.empty:
                        # Extract weights in the order corresponding to source_node_ids
                        # Ensure 'source' column is used for indexing for correct alignment
                        weights_aligned = valid_incoming_edges.set_index('source').loc[list(source_results_map.keys())]['weight'].values
                        x_values = np.array(list(source_results_map.values()))
                        
                        b = self.nodes.loc[self.nodes['id'] == node_id, 'bias'].values[0]
                        
                        # Dot product of inputs (x) and weights (w) plus bias
                        output = ReLU(np.dot(x_values, weights_aligned) + b)
                        self.results.loc[self.results['node id'] == node_id, 'result'] = output
            
            # Extract and return output values
            output_node_ids = self.nodes.loc[self.nodes['type'] == 'output', 'id'].values
            outputs = self.results.loc[self.results['node id'].isin(output_node_ids), 'result'].values
            
            # Convert to list and replace None/NaN values with 0.0 for consistency
            final_outputs = [0.0 if v is None or pd.isna(v) else float(v) for v in outputs]
            return final_outputs

### Evolution

This section defines the Evolution class, which orchestrates the evolutionary process for optimizing neural network chromosomes. It includes methods for fitness evaluation (using the CartPole environment), genetic operators (selection, crossover, mutation), and tracking evolutionary progress.

In [ ]:
"""
This module defines the Evolution class, which orchestrates the evolutionary
process for optimizing neural network chromosomes, typically applied to
reinforcement learning tasks like CartPole. It includes methods for fitness
evaluation, genetic operators (selection, crossover, mutation), and tracking
evolutionary progress.
"""

class Evolution:
    """
    Manages the evolutionary algorithm, including population generation,
    fitness evaluation, parent selection, crossover, and mutation to evolve
    neural network chromosomes.
    """
    def __init__(self, population_size: int = 20, inputs: int = 4, outputs: int = 1, hidden_nodes: int = 15, connectivity_ratio: float = 0.5):
        """
        Initializes the evolutionary process parameters and creates the initial population.

        Args:
            population_size (int, optional): Number of chromosomes in each generation. Defaults to 20.
            inputs (int, optional): Number of input nodes for each neural network in the population. Defaults to 4.
            outputs (int, optional): Number of output nodes for each neural network in the population. Defaults to 1.
            hidden_nodes (int, optional): Initial number of hidden nodes for newly created chromosomes. Defaults to 15.
            connectivity_ratio (float, optional): Initial density of connections for new chromosomes. Defaults to 0.5.
        """
        self.population_size = population_size
        self.inputs = inputs
        self.outputs = outputs
        self.hidden_nodes = hidden_nodes
        self.connectivity_ratio = connectivity_ratio
        self.generation = 0
        self.population: list[Chromosome] = []
        self.best_fitness = -float('inf') # Initialize with negative infinity
        self.best_chromosome: Chromosome | None = None
        self.fitness_history: list[dict] = []
        
        # Create initial population
        for i in range(self.population_size):
            chromosome = Chromosome(
                id=i,
                inputs=self.inputs,
                outputs=self.outputs,
                hidden_nodes=self.hidden_nodes,
                connectivity_ratio=self.connectivity_ratio,
                generation=self.generation
            )
            self.population.append(chromosome)
    
    def _validate_network(self, chromosome: Chromosome) -> bool:
        """
        Validates the structure of a neural network chromosome.

        Checks for basic validity like:
        - All input nodes having at least one outgoing connection.
        - The network being runnable with the expected input size.

        Args:
            chromosome (Chromosome): The chromosome to validate.

        Returns:
            bool: True if the network structure is considered valid, False otherwise.
        """
        input_nodes = chromosome.get_input_nodes()['id'].values
        
        if len(input_nodes) != self.inputs:
            print(f"Validation Error: Chromosome {chromosome.id} has {len(input_nodes)} input nodes, expected {self.inputs}.")
            return False

        for input_id in input_nodes:
            if chromosome.get_edges_by_source(input_id).empty:
                print(f"Validation Error: Input node {input_id} in chromosome {chromosome.id} has no outgoing connections.")
                return False
                
        # Check network can handle expected input size by running a dummy input
        try:
            test_input = np.zeros(self.inputs)
            chromosome.NN.run(test_input)
            return True
        except ValueError as e:
            print(f"Network validation failed for chromosome {chromosome.id} due to input size mismatch: {e}")
            return False
        except Exception as e:
            print(f"Network validation failed for chromosome {chromosome.id} due to runtime error: {e}")
            return False
    
    def evaluate_fitness(self, chromosome: Chromosome, render: bool = False) -> float:
        """
        Evaluates the fitness of a chromosome by running it in the CartPole-v1 environment.
        The fitness score is the total reward accumulated during an episode.

        Args:
            chromosome (Chromosome): The chromosome (neural network) to evaluate.
            render (bool, optional): If True, the environment will be rendered visually. Defaults to False.

        Returns:
            float: The total reward (fitness score) achieved in the CartPole environment.
        """
        # A network with invalid structure gets minimum fitness
        if not self._validate_network(chromosome):
            # print(f"Warning: Invalid network structure in chromosome {chromosome.id}, returning fitness 0.")
            return 0.0 # Return minimum fitness for invalid networks
        
        # Initialize the CartPole environment
        env = gym.make('CartPole-v1', render_mode='human' if render else None)
        observation, _ = env.reset()
        
        done = False
        total_reward = 0
        max_steps = 500  # Maximum episode length for CartPole-v1
        
        while not done and total_reward < max_steps:
            # Use neural network to determine action
            action = self._get_action(chromosome, observation)
            
            # Take step in environment
            observation, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            total_reward += reward
        
        env.close()
        return float(total_reward)
    
    def _get_action(self, chromosome: Chromosome, observation: np.ndarray) -> int:
        """
        Determines the action to take in the environment using the neural network.

        For CartPole, the observation values are already in a reasonable range,
        so no explicit normalization is applied here. The network's single
        output is thresholded to 0 or 1.

        Args:
            chromosome (Chromosome): The neural network chromosome.
            observation (np.ndarray): The current state observation from the environment.

        Returns:
            int: The chosen action (0 for left, 1 for right).
        """
        # CartPole observation values are typically small, direct use often works
        # If necessary, further normalization could be added here (e.g., scaling to -1 to 1)
        norm_obs = observation.astype(float) # Ensure float type for NN

        output = chromosome.NN.run(norm_obs)
        
        # Convert output to discrete action (0 or 1)
        # Assuming a single output node for binary classification
        if output: # Check if output list is not empty
            return 1 if output[0] > 0.5 else 0
        return 0 # Default action if network somehow produces no output
    
    def evaluate_population(self) -> list[tuple[Chromosome, float]]:
        """
        Evaluates the fitness of every chromosome in the current population.
        Sorts the population by fitness in descending order and updates the
        overall best chromosome and fitness history.

        Returns:
            list[tuple[Chromosome, float]]: A list of (chromosome, fitness) tuples,
                                            sorted by fitness from highest to lowest.
        """
        fitness_scores = []
        
        # Use tqdm for a progress bar during evaluation
        for chromosome in tqdm(self.population, desc=f"Generation {self.generation}"):
            fitness = self.evaluate_fitness(chromosome)
            fitness_scores.append((chromosome, fitness))
            
        # Sort by fitness (descending)
        fitness_scores.sort(key=lambda x: x[1], reverse=True)
        
        # Update best chromosome if a new best is found
        if fitness_scores[0][1] > self.best_fitness:
            self.best_fitness = fitness_scores[0][1]
            self.best_chromosome = fitness_scores[0][0]
        
        # Record average and max fitness for history tracking
        avg_fitness = sum(score for _, score in fitness_scores) / len(fitness_scores)
        self.fitness_history.append({
            'generation': self.generation,
            'max_fitness': fitness_scores[0][1],
            'avg_fitness': avg_fitness
        })
        
        return fitness_scores
    
    def select_parents(self, fitness_scores: list[tuple[Chromosome, float]], selection_method: str = 'tournament', tournament_size: int = 3) -> list[Chromosome]:
        """
        Selects parent chromosomes from the population based on their fitness.

        Args:
            fitness_scores (list[tuple[Chromosome, float]]): A list of (chromosome, fitness) tuples.
                                                            Expected to be sorted by fitness.
            selection_method (str, optional): The method to use for selection ('tournament' or 'roulette').
                                              Defaults to 'tournament'.
            tournament_size (int, optional): Number of individuals in each tournament (for 'tournament' method).
                                             Defaults to 3.

        Returns:
            list[Chromosome]: A list of selected parent chromosomes, typically `self.population_size` in length.
        """
        parents = []
        
        if selection_method == 'tournament':
            for _ in range(self.population_size):
                # Ensure tournament_size doesn't exceed available population
                actual_tournament_size = min(tournament_size, len(fitness_scores))
                if actual_tournament_size == 0: # Handle empty fitness_scores
                    break
                tournament = random.sample(fitness_scores, actual_tournament_size)
                winner = max(tournament, key=lambda x: x[1])
                parents.append(winner[0])
                
        elif selection_method == 'roulette':
            total_fitness = sum(score for _, score in fitness_scores)
            
            # Handle cases where all fitnesses are zero or negative
            if total_fitness <= 0: # If all are zero or negative, give equal chance, or pick best
                # Assign small positive fitness to all if total_fitness is 0 or negative
                # to allow selection. Or, more simply, just pick randomly or elite.
                if len(fitness_scores) > 0:
                    # If all fitness is zero, pick randomly. If negative, might still be relative.
                    # For robust roulette, all fitnesses should be non-negative.
                    # Add a small offset if all are zero to ensure pickability.
                    min_fitness = min(score for _, score in fitness_scores) if fitness_scores else 0
                    if min_fitness < 0:
                        adjusted_scores = [(chrom, score - min_fitness + 1) for chrom, score in fitness_scores]
                        total_fitness = sum(score for _, score in adjusted_scores)
                    elif total_fitness == 0 and len(fitness_scores) > 0:
                        adjusted_scores = [(chrom, 1) for chrom, _ in fitness_scores]
                        total_fitness = len(fitness_scores)
                    else:
                        adjusted_scores = fitness_scores # Should not happen if total_fitness <= 0 but also > 0
                else:
                    return [] # No parents if no fitness scores
            else:
                adjusted_scores = fitness_scores

            for _ in range(self.population_size):
                pick = random.uniform(0, total_fitness)
                current = 0
                for chromosome, score in adjusted_scores:
                    current += score
                    if current > pick:
                        parents.append(chromosome)
                        break
                # Fallback in case floating point issues or an edge case
                if not parents or len(parents) < _ + 1:
                    parents.append(fitness_scores[0][0]) # Add the best if selection fails

        else:
            raise ValueError(f"Unknown selection method: {selection_method}")
        
        return parents
    
    def crossover(self, parent1: Chromosome, parent2: Chromosome) -> Chromosome:
        """
        Performs crossover between two parent chromosomes to create a child chromosome.
        The crossover strategy combines nodes and edges from both parents.
        Prioritizes combining edges based on target nodes.

        Args:
            parent1 (Chromosome): The first parent chromosome.
            parent2 (Chromosome): The second parent chromosome.

        Returns:
            Chromosome: A new child chromosome resulting from the crossover.
        """
        all_nodes_list = []
        # Start with all nodes from parent1
        all_nodes_list.extend(parent1.nodes.to_dict('records'))
        
        # Add nodes from parent2 that are not in parent1
        parent1_node_ids = set(parent1.nodes['id'].values)
        for _, node_row in parent2.nodes.iterrows():
            if node_row['id'] not in parent1_node_ids:
                all_nodes_list.append(node_row.to_dict())

        # Create a combined node DataFrame, ensure unique IDs after copying (especially new nodes from parent2)
        combined_nodes_df = pd.DataFrame(all_nodes_list).drop_duplicates(subset=['id']).reset_index(drop=True)

        # Re-index node IDs to be contiguous and start from 0 for the child
        # This is crucial for keeping node IDs consistent and avoiding huge sparse IDs.
        # Create a mapping from old ID to new ID
        old_to_new_node_id_map = {old_id: new_id for new_id, old_id in enumerate(combined_nodes_df['id'].unique())}
        combined_nodes_df['id'] = combined_nodes_df['id'].map(old_to_new_node_id_map)

        edge_records = []
        
        # Group edges by target node for easier handling
        p1_edges_by_target = parent1.edges.copy().groupby('target')
        p2_edges_by_target = parent2.edges.copy().groupby('target')
        
        # Get all target nodes from both parents, and map them to new IDs
        all_targets_old_ids = set(parent1.edges['target'].unique()) | set(parent2.edges['target'].unique())
        all_targets_new_ids = {old_to_new_node_id_map.get(old_id, old_id) for old_id in all_targets_old_ids} # Handle case where target node might not have been in combined_nodes_df

        # For each target node, randomly choose which parent's incoming edges to use
        # or combine them carefully
        for old_target_id in all_targets_old_ids:
            new_target_id = old_to_new_node_id_map.get(old_target_id, old_target_id)
            
            # Simple choice: 50/50 probability to pick parent's edges for this target
            chosen_parent = random.choice([1, 2])

            source_edges_to_add = pd.DataFrame(columns=parent1.edges.columns) # Temporary DF
            
            if chosen_parent == 1 and old_target_id in p1_edges_by_target.groups:
                source_edges_to_add = pd.concat([source_edges_to_add, p1_edges_by_target.get_group(old_target_id)])
            elif chosen_parent == 2 and old_target_id in p2_edges_by_target.groups:
                source_edges_to_add = pd.concat([source_edges_to_add, p2_edges_by_target.get_group(old_target_id)])

            for _, edge in source_edges_to_add.iterrows():
                new_source_id = old_to_new_node_id_map.get(edge['source'], edge['source'])
                
                # Check if the source node exists in the child's new combined_nodes_df
                # This should prevent adding edges to non-existent nodes in the child.
                if new_source_id in combined_nodes_df['id'].values and new_target_id in combined_nodes_df['id'].values:
                    edge_records.append({
                        'source': new_source_id,
                        'target': new_target_id,
                        'weight': edge['weight'],
                        'enabled': edge['enabled']
                    })
        
        # Create child edges DataFrame, dropping duplicates based on (source, target) pairs
        child_edges_df = pd.DataFrame(edge_records).drop_duplicates(subset=['source', 'target']).reset_index(drop=True)
        # Assign new sequential IDs to edges
        child_edges_df['id'] = range(len(child_edges_df))

        # Ensure essential nodes (inputs/outputs) exist and are connected
        # Input nodes check
        child_input_nodes_ids = combined_nodes_df[combined_nodes_df['type'] == 'input']['id'].values
        for expected_input_idx in range(self.inputs):
            # Check if we have enough input nodes. If not, add dummy ones or from parents (if available).
            if expected_input_idx >= len(child_input_nodes_ids):
                # Find an input node from parents that hasn't been added yet
                candidate_input_nodes_p1 = parent1.get_input_nodes().sort_values('id').iloc[expected_input_idx - len(child_input_nodes_ids):]
                candidate_input_nodes_p2 = parent2.get_input_nodes().sort_values('id').iloc[expected_input_idx - len(child_input_nodes_ids):]
                
                new_input_node_data = None
                if not candidate_input_nodes_p1.empty:
                    new_input_node_data = candidate_input_nodes_p1.iloc[0].to_dict()
                elif not candidate_input_nodes_p2.empty:
                    new_input_node_data = candidate_input_nodes_p2.iloc[0].to_dict()
                
                if new_input_node_data:
                    # Map old ID to new ID, or assign a truly new one
                    new_input_node_data['id'] = old_to_new_node_id_map.get(new_input_node_data['id'], combined_nodes_df['id'].max() + 1 if not combined_nodes_df.empty else 0)
                    if new_input_node_data['id'] in combined_nodes_df['id'].values: # If ID clashes, find next available
                         new_input_node_data['id'] = combined_nodes_df['id'].max() + 1 if not combined_nodes_df.empty else 0

                    combined_nodes_df = pd.concat([combined_nodes_df, pd.DataFrame([new_input_node_data])], ignore_index=True)
                    child_input_nodes_ids = combined_nodes_df[combined_nodes_df['type'] == 'input']['id'].values # Refresh list
                else:
                    # If no candidate, add a completely new input node
                    new_input_id = combined_nodes_df['id'].max() + 1 if not combined_nodes_df.empty else 0
                    new_input_node_data = {'id': new_input_id, 'type': 'input', 'bias': 0.0, 'birth_generation': self.generation + 1, 'division_count': 0}
                    combined_nodes_df = pd.concat([combined_nodes_df, pd.DataFrame([new_input_node_data])], ignore_index=True)
                    child_input_nodes_ids = combined_nodes_df[combined_nodes_df['type'] == 'input']['id'].values # Refresh list

        # Output nodes check (similar logic to input nodes)
        child_output_nodes_ids = combined_nodes_df[combined_nodes_df['type'] == 'output']['id'].values
        for expected_output_idx in range(self.outputs):
            if expected_output_idx >= len(child_output_nodes_ids):
                candidate_output_nodes_p1 = parent1.get_output_nodes().sort_values('id').iloc[expected_output_idx - len(child_output_nodes_ids):]
                candidate_output_nodes_p2 = parent2.get_output_nodes().sort_values('id').iloc[expected_output_idx - len(child_output_nodes_ids):]
                
                new_output_node_data = None
                if not candidate_output_nodes_p1.empty:
                    new_output_node_data = candidate_output_nodes_p1.iloc[0].to_dict()
                elif not candidate_output_nodes_p2.empty:
                    new_output_node_data = candidate_output_nodes_p2.iloc[0].to_dict()

                if new_output_node_data:
                    new_output_node_data['id'] = old_to_new_node_id_map.get(new_output_node_data['id'], combined_nodes_df['id'].max() + 1 if not combined_nodes_df.empty else 0)
                    if new_output_node_data['id'] in combined_nodes_df['id'].values:
                         new_output_node_data['id'] = combined_nodes_df['id'].max() + 1 if not combined_nodes_df.empty else 0
                    combined_nodes_df = pd.concat([combined_nodes_df, pd.DataFrame([new_output_node_data])], ignore_index=True)
                    child_output_nodes_ids = combined_nodes_df[combined_nodes_df['type'] == 'output']['id'].values # Refresh list
                else:
                    new_output_id = combined_nodes_df['id'].max() + 1 if not combined_nodes_df.empty else 0
                    new_output_node_data = {'id': new_output_id, 'type': 'output', 'bias': np.random.uniform(-1, 1), 'birth_generation': self.generation + 1, 'division_count': 0}
                    combined_nodes_df = pd.concat([combined_nodes_df, pd.DataFrame([new_output_node_data])], ignore_index=True)
                    child_output_nodes_ids = combined_nodes_df[combined_nodes_df['type'] == 'output']['id'].values # Refresh list
        
        # Ensure all input nodes have at least one outgoing connection
        for input_id in combined_nodes_df[combined_nodes_df['type'] == 'input']['id'].values:
            if not any(child_edges_df['source'] == input_id):
                hidden_nodes_in_child = combined_nodes_df[combined_nodes_df['type'] == 'hidden']
                target_id = None
                if not hidden_nodes_in_child.empty:
                    target_id = hidden_nodes_in_child.sample(1).iloc[0]['id']
                else: # Fallback to output nodes if no hidden
                    output_nodes_in_child = combined_nodes_df[combined_nodes_df['type'] == 'output']
                    if not output_nodes_in_child.empty:
                        target_id = output_nodes_in_child.sample(1).iloc[0]['id']
                
                if target_id is not None:
                    new_edge_id = child_edges_df['id'].max() + 1 if not child_edges_df.empty else 0
                    new_edge_row = pd.DataFrame([{
                        'id': new_edge_id,
                        'source': input_id,
                        'target': target_id,
                        'weight': np.random.uniform(-1, 1),
                        'enabled': True
                    }])
                    child_edges_df = pd.concat([child_edges_df, new_edge_row], ignore_index=True)
                # else: No suitable target, network might be problematic, but we proceed
        
        # Ensure all output nodes have at least one incoming connection
        for output_id in combined_nodes_df[combined_nodes_df['type'] == 'output']['id'].values:
            if not any(child_edges_df['target'] == output_id):
                hidden_nodes_in_child = combined_nodes_df[combined_nodes_df['type'] == 'hidden']
                source_id = None
                if not hidden_nodes_in_child.empty:
                    source_id = hidden_nodes_in_child.sample(1).iloc[0]['id']
                else: # Fallback to input nodes if no hidden
                    input_nodes_in_child = combined_nodes_df[combined_nodes_df['type'] == 'input']
                    if not input_nodes_in_child.empty:
                        source_id = input_nodes_in_child.sample(1).iloc[0]['id']
                
                if source_id is not None:
                    new_edge_id = child_edges_df['id'].max() + 1 if not child_edges_df.empty else 0
                    new_edge_row = pd.DataFrame([{
                        'id': new_edge_id,
                        'source': source_id,
                        'target': output_id,
                        'weight': np.random.uniform(-1, 1),
                        'enabled': True
                    }])
                    child_edges_df = pd.concat([child_edges_df, new_edge_row], ignore_index=True)
                # else: No suitable source, network might be problematic, but we proceed

        # Create new chromosome ID for the child
        # Ensure child ID is unique in context of the population, if needed later for tracking
        new_id = self.id_counter.next_id() if hasattr(self, 'id_counter') else max(parent1.id, parent2.id) + 1
        
        # Create and return the child chromosome
        child = Chromosome(
            id=new_id,
            nodes_df=combined_nodes_df,
            edges_df=child_edges_df,
            generation=self.generation + 1
        )
        
        return child
    
    def mutate(self, chromosome: Chromosome, mutation_rate: float = 0.1, weight_mutation_scale: float = 0.5, bias_mutation_scale: float = 0.2) -> Chromosome:
        """
        Mutates a chromosome by altering weights, biases, and potentially its structure.

        Mutation types include:
        - Weight mutation: Random noise added to edge weights.
        - Bias mutation: Random noise added to node biases (excluding input nodes).
        - Edge enablement/disablement: Toggles 'enabled' status of some edges.
        - Add new edge: Creates a new connection between existing nodes (cycle-checked).
        - Add new hidden node: Inserts a new hidden node and connects it.

        Args:
            chromosome (Chromosome): The chromosome to be mutated.
            mutation_rate (float, optional): Base probability for each mutation type to occur. Defaults to 0.1.
            weight_mutation_scale (float, optional): Standard deviation for normal distribution when mutating weights. Defaults to 0.5.
            bias_mutation_scale (float, optional): Standard deviation for normal distribution when mutating biases. Defaults to 0.2.

        Returns:
            Chromosome: The mutated chromosome.
        """
        # Mutate edge weights
        for index, edge in chromosome.edges.iterrows():
            if random.random() < mutation_rate:
                new_weight = edge['weight'] + np.random.normal(0, weight_mutation_scale)
                chromosome.update_edge_weight(edge['id'], new_weight)
        
        # Mutate node biases (except input nodes)
        non_input_nodes = chromosome.nodes[chromosome.nodes['type'] != 'input']
        for index, node in non_input_nodes.iterrows():
            if random.random() < mutation_rate:
                new_bias = node['bias'] + np.random.normal(0, bias_mutation_scale)
                chromosome.update_node_bias(node['id'], new_bias)
        
        # Structural mutation: enable/disable edges
        if random.random() < mutation_rate * 0.5:
            if not chromosome.edges.empty:
                # Select a small subset of edges to potentially toggle
                num_edges_to_toggle = max(1, int(len(chromosome.edges) * 0.1))
                edges_to_consider = chromosome.edges.sample(min(num_edges_to_toggle, len(chromosome.edges)))
                
                for _, edge_to_toggle in edges_to_consider.iterrows():
                    edge_id = edge_to_toggle['id']
                    if edge_to_toggle['enabled']:
                        # Simple heuristic: only disable if target node has other incoming enabled edges
                        # This avoids completely isolating a node if it's crucial.
                        incoming_edges_to_target = chromosome.get_edges_by_target(edge_to_toggle['target'])
                        if len(incoming_edges_to_target[incoming_edges_to_target['enabled'] == True]) > 1:
                            chromosome.disable_edge(edge_id)
                    else:
                        # Re-enable disabled edge, but check for cycle creation first
                        if not chromosome.would_create_cycle(edge_to_toggle['source'], edge_to_toggle['target']):
                            idx = chromosome.edges.index[chromosome.edges['id'] == edge_id].tolist()
                            if idx:
                                chromosome.edges.loc[idx[0], 'enabled'] = True
        
        # Structural mutation: add new edge
        if random.random() < mutation_rate * 0.3:
            node_ids = chromosome.nodes['id'].values
            if len(node_ids) > 1:
                for _ in range(5): # Try up to 5 times to find a valid connection
                    source = random.choice(node_ids)
                    target = random.choice(node_ids)
                    
                    # Ensure valid connection types (e.g., no output to input, no self-loops)
                    source_type = chromosome.nodes[chromosome.nodes['id'] == source]['type'].values[0]
                    target_type = chromosome.nodes[chromosome.nodes['id'] == target]['type'].values[0]
                    
                    if source_type == 'output' or target_type == 'input' or source == target:
                        continue
                    
                    # Also, avoid adding an edge that already exists and is enabled
                    existing_edge = chromosome.get_edges_between(source, target)
                    if not existing_edge.empty and existing_edge.iloc[0]['enabled']:
                        continue

                    if chromosome.add_edge(source=source, target=target, weight=np.random.uniform(-1, 1)) is not None:
                        break # Successfully added an edge
        
        # Structural mutation: add new hidden node
        if random.random() < mutation_rate * 0.1:
            new_node_id = chromosome.add_node(
                node_type='hidden',
                bias=np.random.uniform(-1, 1),
                birth_generation=self.generation
            )
            
            if new_node_id is not None:
                # Try to connect it to the network
                existing_nodes = chromosome.nodes[chromosome.nodes['id'] != new_node_id]['id'].values
                
                # Connect from a random existing node (if suitable)
                if len(existing_nodes) > 0:
                    for _ in range(3): # Try a few times
                        source_id = random.choice(existing_nodes)
                        source_type = chromosome.nodes[chromosome.nodes['id'] == source_id]['type'].values[0]
                        if source_type != 'output':
                            chromosome.add_edge(source=source_id, target=new_node_id, weight=np.random.uniform(-1, 1))
                            break
                
                # Connect to a random existing node (if suitable)
                if len(existing_nodes) > 0:
                    for _ in range(3): # Try a few times
                        target_id = random.choice(existing_nodes)
                        target_type = chromosome.nodes[chromosome.nodes['id'] == target_id]['type'].values[0]
                        if target_type != 'input':
                            chromosome.add_edge(source=new_node_id, target=target_id, weight=np.random.uniform(-1, 1))
                            break
        
        return chromosome
    
    def evolve(self, generations: int = 10, mutation_rate: float = 0.2, selection_method: str = 'tournament') -> Chromosome:
        """
        Runs the main evolutionary loop for a specified number of generations.

        In each generation, it performs:
        1. Fitness evaluation of the current population.
        2. Parent selection based on fitness.
        3. Crossover to create offspring.
        4. Mutation of offspring.
        5. Replacement of the old population with the new one (plus elitism).

        Args:
            generations (int, optional): The total number of generations to run the evolution. Defaults to 10.
            mutation_rate (float, optional): The base probability for mutation operations. Defaults to 0.2.
            selection_method (str, optional): The method for parent selection ('tournament' or 'roulette').
                                              Defaults to 'tournament'.

        Returns:
            Chromosome: The best chromosome found across all generations.
        """
        # A simple ID counter for new chromosomes generated during evolution
        class IDCounter:
            def __init__(self, start_id: int):
                self._current_id = start_id
            def next_id(self):
                self._current_id += 1
                return self._current_id - 1
        self.id_counter = IDCounter(max(c.id for c in self.population) + 1 if self.population else 0)


        for gen in range(generations):
            self.generation = gen # Update current generation
            
            # 1. Evaluate current population
            fitness_scores = self.evaluate_population()
            
            # Print generation statistics
            best_fitness_gen = fitness_scores[0][1]
            avg_fitness_gen = sum(score for _, score in fitness_scores) / len(fitness_scores)
            print(f"Generation {self.generation}: Best fitness = {best_fitness_gen}, Avg fitness = {avg_fitness_gen:.2f}")
            
            # 2. Select parents
            parents = self.select_parents(fitness_scores, selection_method)
            
            # 3. Create new population
            new_population: list[Chromosome] = []
            
            # Elitism: Keep the single best chromosome from the previous generation
            new_population.append(fitness_scores[0][0])
            
            # Create offspring until the population size is reached
            while len(new_population) < self.population_size:
                parent1 = random.choice(parents)
                parent2 = random.choice(parents)
                
                # Ensure parents are distinct if there's more than one parent available
                attempts = 0
                while parent2.id == parent1.id and len(parents) > 1 and attempts < 10:
                    parent2 = random.choice(parents)
                    attempts += 1
                
                # Crossover to create a child
                child = self.crossover(parent1, parent2)
                
                # Mutate the child
                child = self.mutate(child, mutation_rate)
                
                # Assign a unique ID to the new child
                child.id = self.id_counter.next_id()
                child.generation = self.generation + 1 # Child belongs to the next generation
                
                new_population.append(child)
            
            # Update population for the next generation
            self.population = new_population
        
        # Return the best chromosome found throughout the entire evolutionary run
        return self.best_chromosome
    
    def visualize_fitness_history(self):
        """
        Visualizes the maximum and average fitness scores over generations.
        Plots a line graph showing the evolution of fitness.
        """
        if not self.fitness_history:
            print("No fitness history to visualize.")
            return

        df = pd.DataFrame(self.fitness_history)
        
        plt.figure(figsize=(10, 6))
        plt.plot(df['generation'], df['max_fitness'], 'b-', label='Max Fitness')
        plt.plot(df['generation'], df['avg_fitness'], 'r-', label='Average Fitness')
        plt.xlabel('Generation')
        plt.ylabel('Fitness')
        plt.title('Fitness History Over Generations')
        plt.legend()
        plt.grid(True)
        plt.show()

This function encapsulates the setup and execution of the entire evolutionary experiment for the CartPole task.


In [ ]:
def run_evolution_experiment():
    """
    Orchestrates and runs a complete evolutionary experiment for the CartPole task.
    Initializes the evolutionary process, runs it for a specified number of generations,
    visualizes the fitness progress, and then tests and visualizes the best performing
    chromosome.
    """
    # Define experiment parameters
    POPULATION_SIZE = 30
    GENERATIONS = 20
    MUTATION_RATE = 0.2
    
    # Initialize the evolutionary algorithm instance
    evolution = Evolution(
        population_size=POPULATION_SIZE,
        inputs=4,  # CartPole observation space has 4 values
        outputs=1,  # CartPole action space has 2 actions (left/right), 1 output is enough (e.g., >0.5 -> right, <=0.5 -> left)
        hidden_nodes=15, # Starting number of hidden nodes
        connectivity_ratio=0.6 # Initial connection density
    )
    
    # Run the evolutionary process
    best_chromosome = evolution.evolve(
        generations=GENERATIONS,
        mutation_rate=MUTATION_RATE,
        selection_method='tournament'
    )
    
    # Visualize the fitness trends over generations
    evolution.visualize_fitness_history()
    
    # Test the best chromosome found with rendering enabled to observe its behavior
    if best_chromosome:
        print(f"\nTesting best chromosome (Generation {best_chromosome.generation}, ID {best_chromosome.id})")
        final_fitness = evolution.evaluate_fitness(best_chromosome, render=True)
        print(f"Final fitness of best chromosome: {final_fitness}")
        
        # Visualize the structure of the best performing neural network
        plt.figure(figsize=(12, 10))
        # Ensure the visualize method does not call plt.show() if an ax is passed
        best_chromosome.visualize() 
        plt.title(f"Best Chromosome: Gen {best_chromosome.generation}, ID {best_chromosome.id}, Fitness {final_fitness:.2f}")
        plt.show()
    else:
        print("No best chromosome found.")
    
    return best_chromosome, evolution

### Division Demo

This section specifically demonstrates the node division functionality of the Chromosome class. It creates a "division model" (a small neural network) and a main chromosome. The division model then dictates which hidden nodes in the main chromosome should divide, showcasing dynamic network growth.

In [ ]:
print("--- Demonstrating Node Division Functionality ---")

# --- Setup the Division Model Chromosome ---
# This Chromosome acts as a separate neural network that decides if other nodes should divide.
# Its inputs are features of a node (age, division count, connection count, node_id).
# Its single output (via a threshold) determines the division decision.
division_model = Chromosome(
    id=999,
    inputs=4,  # [node_age, node_division_count, node_connection_count, node_id]
    outputs=1, # [should_divide_probability]
    hidden_nodes=5 # The Chromosome constructor expects 'hidden_nodes' as an int, not a list of layers.
)

# Randomize ALL of the weights in the division model with a wider range
# This makes the division decision more dynamic and less predictable for testing.
for _, edge in division_model.edges.iterrows():
    division_model.update_edge_weight(edge['id'], np.random.uniform(-1.5, 1.5))

# Set a moderate bias for the output node
# This can influence the baseline 'propensity' to divide.
output_nodes = division_model.get_output_nodes()
for _, node in output_nodes.iterrows():
    division_model.update_node_bias(node['id'], 0.2) # A slightly positive bias

# --- Setup the Main Chromosome to be Evolved/Divided ---
# This is the neural network whose structure will be dynamically modified.
chromosome = Chromosome(
    id=1, 
    inputs=3, 
    outputs=2, 
    hidden_nodes=10 # The Chromosome constructor expects 'hidden_nodes' as an int, not a list of layers.
) 

# --- Define a Custom Bias Function for Daughter Nodes ---
# This function determines the bias of a newly created daughter node based on its parent's bias.
# A wider random range is used here to increase variability in daughter node biases.
def my_bias_function(parent_bias: float) -> float:
    """
    Calculates the bias for a daughter node based on its parent's bias.
    Introduces a random perturbation to the parent's bias.

    Args:
        parent_bias (float): The bias of the parent node.

    Returns:
        float: The calculated bias for the daughter node.
    """
    return parent_bias + np.random.uniform(-1.0, 1.0)  # Wider range of variability

# --- Run the Node Division Evaluation ---
# This is where the division_model evaluates each hidden node in 'chromosome'
# and performs division if the criteria are met.
print(f"Initial chromosome {chromosome.id} has {len(chromosome.nodes)} nodes.")
new_nodes = chromosome.evaluate_all_nodes_for_division(
    division_model, 
    bias_func=my_bias_function
)
print(f"After division evaluation, chromosome {chromosome.id} now has {len(chromosome.nodes)} nodes.")
print(f"Newly created nodes during this round of division: {new_nodes}")


# --- Visualize the Network After Division ---
# Shows the updated structure of the main chromosome.
plt.figure(figsize=(12, 10))
chromosome.visualize()
plt.title(f"Chromosome {chromosome.id} after node division (Total nodes: {len(chromosome.nodes)})")
plt.show()

# --- Optional: Detailed Testing of Division Thresholds ---
def test_thresholds():
    """
    Performs multiple division evaluations for each hidden node to observe
    the probabilistic nature of the division decision based on the
    division_model and inherent randomness.
    """
    print("\n--- Testing division model with randomized inputs and outputs (multiple runs per node) ---")
    division_counts = {"will_divide": 0, "wont_divide": 0}
    
    # Get the IDs of hidden nodes AFTER the initial division round
    # This ensures we test nodes that actually exist in the current chromosome state.
    hidden_nodes_current_ids = chromosome.get_hidden_nodes()['id'].values
    
    if len(hidden_nodes_current_ids) == 0:
        print("No hidden nodes to test for division.")
        return

    for node_id in hidden_nodes_current_ids:
        results_for_node = []
        for _ in range(3):  # Test each node 3 times to see variability
            result = chromosome.evaluate_division(node_id, division_model)
            results_for_node.append(result)
            
            if result:
                division_counts["will_divide"] += 1
            else:
                division_counts["wont_divide"] += 1
        
        print(f"Node {node_id}: division results from 3 tests = {results_for_node}")
    
    # Print overall statistics
    total_tests = division_counts["will_divide"] + division_counts["wont_divide"]
    divide_percentage = (division_counts["will_divide"] / total_tests) * 100 if total_tests > 0 else 0
    print(f"\nOverall division statistics across all test runs:")
    print(f"Will divide: {division_counts['will_divide']} times ({divide_percentage:.1f}%)")
    print(f"Won't divide: {division_counts['wont_divide']} times ({100-divide_percentage:.1f}%)")

# Uncomment the line below to run the detailed threshold tests
test_thresholds()

This cell executes the main evolutionary algorithm, evolving a population of neural networks to solve the CartPole problem. It will print progress, visualize fitness history, and finally test and display the best-performing network.


In [ ]:
print("\n--- Running the Full Evolutionary Experiment (CartPole) ---")
best_solution, evo = run_evolution_experiment()